# Training and Modelling for the TS based Cov-Matrix Picker

In [26]:
import torch, os
import pandas as pd
import numpy as np
import pickle
from math import factorial as fac
import matplotlib
from random import sample 
import matplotlib.pyplot as plt 
from itertools import combinations as combs
from numpy import random as npr
import torch.nn as nn

## Support Functions

In [2]:
def nChoosek(n,k):
    return fac(n) // fac(r) // fac(n-r)

def which_mat(TTS, choice, n = [5, 5*3, 5*5, 5*9, 5*13], p = [0], channel_use = 1):
    """
    Funciton which can be easilly calle din order to implament which_mat as part of a larger function
    """
    return get_mat(TTS, n[choice[0]], p[0], channel_use)

def get_mat(TTS, n,p = 0, channel_use = 1):
    """
    Funciton that takes in 
    TTS Tesnor of Time Serieses, and given 
    n number of period to use and
    p  last index period to use, 
    return a covariance matrix of the data for that period in terms of the values in chanel channel_use
    """
    
    d1, d2 = TTS[0], TTS[1]
    #insert function to partition TTS correctly
    
    #creating cova matrix estimation desired
    d = [d1, d1]
    return pd.DataFrame({"S%d" %i: d for i,d in  enumerate(d)}).cov()


def cov_matrix_loss(A,B, type_use = 1):
    """
    Get distance between matracies based either on component wise distance or using eigenvalues
    """
    if type_use == 0:
        ind = np.triu_indices(2)
        loss = nn.MSELoss()
        return loss(A[ind], B[ind]) 
    
    elif type_use == 1:
        return torch.sqrt(torch.sum(torch.pow(torch.log(torch.eig(A-B))),2))
    

## NN Models

In [15]:
class TS_based_simple(nn.Module):
    def __init__(self, num_outs,
                 kernel = 5, in_channels = 4,  hidden_size = 128):
        """
        
        
        """
        super(TS_based_simple, self).__init__()
        
        pads = int(kernel // 2)
        
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = in_channels, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads ),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = int(hidden_size/2), 
                      kernel_size = int(kernel-2), stride = 2, padding = int(pads-1) ),
            nn.BatchNorm1d(int(hidden_size/2)),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/2), out_channels = int(hidden_size/(2**2)), 
                      kernel_size = int(kernel-2), stride = 2, padding = (pads-1) ),
            nn.BatchNorm1d(int(hidden_size/(2**2))),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/(2**2)) , out_channels = num_outs, 
                      kernel_size = int(kernel-2), padding = int(pads-1)),
            nn.BatchNorm1d(num_outs),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.picker = nn.Linear(in_features = num_outs, out_features = num_outs)
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker(self.cl)
        

class TS_based_wfactors(nn.Module):
    def __init__(self, num_outs, kernel = 5, channels_in =4, hidden_size = 128):
        """
        
        
        """
        super(TS_based_wfactors, self).__init__()
        
        pads  = int(kernel//2)
        
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = channels_in, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads ),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = int(hidden_size/2), 
                      kernel_size = int(kernel-2), stride = 2, padding = int(pads-1) ),
            nn.BatchNorm1d(int(hidden_size/2)),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/2), out_channels = int(hidden_size/(2**2)), 
                      kernel_size = int(kernel-2), stride = 2, padding = int(pads-1)),
            nn.BatchNorm1d(int(hidden_size/(2**2))),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/(2**2)) , out_channels = num_outs, 
                      kernel_size = int(kernel-2), padding = int(pads-1)),
            nn.BatchNorm1d(num_outs),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.picker = nn.Linear(in_channels = num_outs , out_channels = num_outs)
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker(self.cl)
        

## Trainer

In [20]:
def train_set(data_use, args):
    
    #how many periods 
    num_periods = list(data_use[0].values())[0].size()[0]
    
    #picking which asset combinations to use
    num_combs = min(args.max_train, nChoosek(args.num_stocks_each,2))
    all_keys = list(data_use[0].keys()) + list(data_use[1].keys())
    which_combs = sample(list(combs(all_keys,2)), num_combs)
    
    
    #how do we pick which periods to test on
    if args.testing_dates == "Random":
        na
    else:
        nada
        
    
    #get training and testing data sets
    
    
    return d_t, d_v

def train(train_d, args, mod_use = None):
    #setting up the packages
    torch.set_num_threads(5)
    npr.seed(args.seed)
    dir_save_to = "results/" + args.dir_name
    if not os.path.exists(dir_save_to):
        os.makedirs(dir_save_to)
    
    
    #type of metric to use for loss:
    l_t = 1
    if arg.loss_type == "L2":
        l_t = 0
    
    
    #which model to implament
    if mod_use is None:
        if args.model_use == "Mod1":
              mod_use = TS_based_simple(num_outs = args.k_size) #args.num_filters, num_colours, arg.num_in_chans)
        elif args.model_use == "Mod1wFactors":
              mod_use = TS_based__wfactors(num_outs = args.k_size) #, args.num_filters, num_colours, arg.num_in_chans)
    
    #setting up the model's optimizaer
    optimizer = torch.optim.Adam(mod_use.parameters(), lr=args.lrn_rate)
    
    #where we are going to gather data
    hist_tr_loss = []
    hist_tt_loss = []
    
    #let's start training
    for epoch in range(arg.num_epochs):
        epoch_loss = []
        
        #let's train
        training_d, validation_d = train_set(train_d, args)
        for input_d, output_d, meta_d in training_d:
            #setup optimizer
            optimizer.zero_grad()
            
            #set up optimization
            output_choice = mod_use(input_d)
            CM_use = which_mat(input_d, output_choice)
            lossing_me = cov_matrix_loss(output_chice, CM_use, l_t)
            
            #optimizing
            lossing_me.backward()
            optimizer.step()
            
            #adding performance
            epoch_loss.append(lossing.data.item())
        
        #printing performance on epoch
        mean_loss = np.mean(epoch_loss)
        hist_tr_loss.append(mean_loss)
        print('Epoch [%d/%d], T Loss: %.4f' % (epoch+1, args.num_epochs, mean_loss))
        
        
        
        #let's validate
        temp_validation = []
        for input_d in validation_d:
            #setup optimizer
            optimizer.zero_grad()
            
            #set up optimization
            output_choice = mod_use(input_d)
            CM_use = which_mat(output_choice)
            lossing_me = cov_matrix_loss(output_chice, CM_use, l_t)
            
            
            #adding performance
            temp_validation.append(lossing.data.item())
        
        hist_tt_loss = []
        #printing performance on epoch
        mean_loss = np.mean(temp_validation)
        hist_tt_loss.append(mean_loss)
        print('Epoch [%d/%d], V Loss: %.4f' % (epoch+1, args.num_epochs, mean_loss))
    
    
    plt.figure()
    plt.plot(hist_tt_loss, "ro-", label="Train")
    plt.plot(hist_tr_loss, "go-", label="Validation")
    plt.legend()
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.savefig(save_dir+"/training_curve.png")

    if args.checkpoint:
        print('Saving model...')
        torch.save(mod_use.state_dict(), args.save_model_as)
    
    return mod_use
        